In [1]:
!pip install -q pandas numpy scikit-learn requests matplotlib plotly

In [2]:
import requests
from datetime import datetime, timedelta
import time
import pandas as pd
import numpy as np
from sklearn.ensemble import IsolationForest, GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import plotly.express as px

In [9]:
from datetime import datetime, timezone
datetime.now(timezone.utc)


datetime.datetime(2025, 10, 12, 13, 55, 28, 864268, tzinfo=datetime.timezone.utc)

In [3]:
# For reproducibility
RANDOM_SEED = 42
LAT = 12.9716 # change to your target latitude
LON = 77.5946 # change to your target longitude
RADIUS_KM = 25
HOURS_HISTORY = 7 * 24 # hours of past data to fetch
FORECAST_HORIZON = 24 # you can set 24 or up to 48

In [5]:
def now_utc():
  return datetime.utcnow()


print("Notebook configured. Change LAT/LON and re-run cells to fetch for another city.")

Notebook configured. Change LAT/LON and re-run cells to fetch for another city.


In [13]:
# 🧠 Step 1: Fetch Live PM2.5 Data — Fixed Version (OpenAQ + Fallback)

## 🔹 Import Libraries
import requests
import pandas as pd
from datetime import datetime, timezone, timedelta

## 🔹 Function: Fetch PM2.5 data from OpenAQ (v3 API)
def fetch_openaq_pm25(lat, lon, radius_km=25000, hours=168, api_key=None):
    """Fetch PM2.5 data from OpenAQ v3 API within a given radius and time window."""
    end = datetime.now(timezone.utc)
    start = end - timedelta(hours=hours)

    headers = {"X-API-Key": api_key} if api_key else {}

    url = (
        f"https://api.openaq.org/v3/measurements?"
        f"parameter=pm25&coordinates={lat},{lon}&radius={radius_km}"
        f"&date_from={start.isoformat()}&date_to={end.isoformat()}&limit=1000"
    )

    response = requests.get(url, headers=headers)
    response.raise_for_status()  # Raise error if request fails

    data = response.json()
    if not data.get("results"):
        return pd.DataFrame()

    df = pd.DataFrame(data["results"])

    # Extract main fields (ensure compatibility)
    if "value" in df.columns and "date" in df.columns:
        df = pd.DataFrame({
            "datetime": pd.to_datetime(df["date"].apply(lambda x: x.get("utc"))),
            "pm25": df["value"]
        })
    elif "datetime" in df.columns:
        df = df.rename(columns={"datetime": "datetime", "value": "pm25"})

    return df.sort_values("datetime").reset_index(drop=True)

## 🔹 Function: Fetch PM2.5 from Open-Meteo (fallback API)

def fetch_open_meteo_pm25(lat, lon, hours=168):
    """Fetch hourly PM2.5 data from Open-Meteo Air Quality API (no key required)."""
    url = f"https://air-quality-api.open-meteo.com/v1/air-quality?latitude={lat}&longitude={lon}&hourly=pm2_5"
    r = requests.get(url)
    r.raise_for_status()
    data = r.json()

    df = pd.DataFrame({
        "datetime": data["hourly"]["time"],
        "pm25": data["hourly"]["pm2_5"]
    })
    df["datetime"] = pd.to_datetime(df["datetime"])

    # Keep only last N hours if needed
    df = df.tail(hours)
    return df.reset_index(drop=True)

## 🔹 Example: Try OpenAQ, else fallback to Open-Meteo
LAT, LON = 12.9716, 77.5946   # Example: Bengaluru
RADIUS_KM = 25000
HOURS_HISTORY = 168  # 7 days
API_KEY = None  # Replace with your OpenAQ key if available

try:
    print("🔹 Fetching PM2.5 data from OpenAQ...")
    pm25 = fetch_openaq_pm25(LAT, LON, RADIUS_KM, HOURS_HISTORY, API_KEY)

    if pm25.empty:
        raise ValueError("Empty data returned from OpenAQ")
    print(f"✅ Retrieved {len(pm25)} records from OpenAQ")

except Exception as e:
    print(f"⚠️ OpenAQ failed ({e}), switching to Open-Meteo fallback...")
    pm25 = fetch_open_meteo_pm25(LAT, LON, HOURS_HISTORY)
    print(f"✅ Retrieved {len(pm25)} records from Open-Meteo fallback")

pm25.head()


## 🔍 Notes:
#- ✅ Fixed **`datetime.utcnow()`** → replaced with `datetime.now(timezone.utc)` (timezone-aware).
#- ✅ Fixed **OpenAQ 410 Error** → updated to new **v3 endpoint**.
#- ✅ Added **fallback to Open-Meteo** (no API key needed).
#- ✅ Added **error handling and clean DataFrame conversion**.

#📘 **Understanding:**
#- `timezone.utc` ensures timestamps are UTC-aware.
#- `raise_for_status()` helps catch HTTP errors early.
#- `try/except` ensures smooth fallback if OpenAQ fails.
#- `pd.to_datetime()` converts string timestamps to datetime objects for time-series modeling.


🔹 Fetching PM2.5 data from OpenAQ...
⚠️ OpenAQ failed (401 Client Error: Unauthorized for url: https://api.openaq.org/v3/measurements?parameter=pm25&coordinates=12.9716,77.5946&radius=25000&date_from=2025-10-05T14:03:17.548298+00:00&date_to=2025-10-12T14:03:17.548298+00:00&limit=1000), switching to Open-Meteo fallback...
✅ Retrieved 120 records from Open-Meteo fallback


,datetime,pm25
0,2025-10-12 00:00:00,51.9
1,2025-10-12 01:00:00,52.0
2,2025-10-12 02:00:00,47.6
3,2025-10-12 03:00:00,35.3
4,2025-10-12 04:00:00,32.1


In [14]:
# 🧠 Step 2: Feature Engineering — Lag Features, Rolling Stats & Anomaly Detection

# Now that we have clean PM2.5 data (from OpenAQ or fallback Open-Meteo), let's transform it for modeling.
#We'll create **lag features**, **rolling window statistics**, and detect **anomalies** using Isolation Forest.


## 🔹 Import Required Libraries

import numpy as np
from sklearn.ensemble import IsolationForest

## 🔹 Function: Add Lag & Rolling Features
def add_lag_features(df, target_col='pm25', max_lag=6, window=3):
    """Add lag and rolling statistical features for time-series modeling."""
    df = df.copy()

    # Create lag features (t-1, t-2, ... t-n)
    for lag in range(1, max_lag + 1):
        df[f'{target_col}_lag_{lag}'] = df[target_col].shift(lag)

    # Rolling mean and std (windowed)
    df[f'{target_col}_roll_mean_{window}'] = df[target_col].rolling(window=window).mean()
    df[f'{target_col}_roll_std_{window}'] = df[target_col].rolling(window=window).std()

    # Drop initial NaNs
    df = df.dropna().reset_index(drop=True)
    return df


## 🔹 Function: Detect Anomalies with Isolation Forest

def detect_anomalies(df, target_col='pm25'):
    """Use Isolation Forest to flag PM2.5 anomalies."""
    model = IsolationForest(contamination=0.05, random_state=42)
    df = df.copy()

    # Fit on PM2.5 values
    df['anomaly_flag'] = model.fit_predict(df[[target_col]])

    # Convert -1 → anomaly, 1 → normal
    df['anomaly_flag'] = df['anomaly_flag'].map({1: 0, -1: 1})
    return df

## 🔹 Apply Feature Engineering

# 1️⃣ Add lag & rolling features
pm25_features = add_lag_features(pm25, target_col='pm25', max_lag=6, window=3)

# 2️⃣ Detect anomalies
pm25_features = detect_anomalies(pm25_features, target_col='pm25')

print(f"✅ Feature-engineered data shape: {pm25_features.shape}")
pm25_features.head()


## 🔍 Notes:
#- **Lag features:** Capture temporal trends (e.g., how past PM2.5 values affect the next).
#- **Rolling mean/std:** Smooth short-term noise and quantify variability.
#- **Isolation Forest:** Identifies unusual pollution spikes or drops automatically.

#📘 **Understanding:**
#- `shift(lag)` aligns past readings for supervised learning.
#- `rolling()` helps the model learn local fluctuations.
#- Anomalies can later be visualized on Streamlit with red markers for pollution spikes.


✅ Feature-engineered data shape: (114, 11)


,datetime,pm25,pm25_lag_1,pm25_lag_2,pm25_lag_3,pm25_lag_4,pm25_lag_5,pm25_lag_6,pm25_roll_mean_3,pm25_roll_std_3,anomaly_flag
0,2025-10-12 06:00:00,30.2,31.8,32.1,35.3,47.6,52.0,51.9,31.366667,1.021437,0
1,2025-10-12 07:00:00,28.5,30.2,31.8,32.1,35.3,47.6,52.0,30.166667,1.650253,0
2,2025-10-12 08:00:00,26.7,28.5,30.2,31.8,32.1,35.3,47.6,28.466667,1.750238,0
3,2025-10-12 09:00:00,25.7,26.7,28.5,30.2,31.8,32.1,35.3,26.966667,1.418920,0
4,2025-10-12 10:00:00,25.4,25.7,26.7,28.5,30.2,31.8,32.1,25.933333,0.680686,0


In [26]:
# ============================================================
# 🧩 STEP 3 + 4: FORECAST PM2.5 AND VISUALIZE RESULTS
# ============================================================

import pandas as pd
import numpy as np
import plotly.graph_objects as go
from sklearn.metrics import mean_absolute_error

# -------------------------------
# Step 3: Generate Forecast
# -------------------------------

# Forecast horizon in hours
FORECAST_HORIZON = 24  # Change to 48 for 48-hour forecast

# Last observed timestamp in your feature dataset
last_ts = pm25_features.index.max()

# Use the fixed recursive_forecast() function to predict next hours
# Make sure to use your trained pipeline variable name (here 'pipeline')
forecast_df = recursive_forecast(
    pipeline,        # your trained model pipeline
    pm25_features,   # feature dataset used for training
    last_ts,         # last observed timestamp
    horizon=FORECAST_HORIZON
)

print(f"✅ Forecast completed! Generated {len(forecast_df)} future PM2.5 predictions.")
display(forecast_df.head())

# -------------------------------
# Step 4a: Prepare Observed + Forecast Data
# -------------------------------

# Use merged dataset if available, otherwise fallback to pm25_features
try:
    obs_recent = merged[['datetime', 'pm25']].set_index('datetime')
except NameError:
    obs_recent = pm25_features.reset_index()[['datetime', 'pm25']].set_index('datetime')

# Take last 48 hours of observed PM2.5
obs_recent = obs_recent.loc[pm25_features.index.max() - pd.Timedelta(hours=48):]

# Ensure forecast_df has datetime index
forecast_df = forecast_df.copy()
forecast_df.index = pd.to_datetime(forecast_df.index)

# Merge observed + forecast for plotting
combined = pd.concat([
    obs_recent.rename(columns={'pm25': 'Observed'}),
    forecast_df.rename(columns={'pm25_pred': 'Forecast'})
], axis=1)

print("✅ Combined observed and forecast data ready for visualization.")
display(combined.tail())

# -------------------------------
# Step 4b: Plot Observed vs Forecast
# -------------------------------

fig = go.Figure()

# Observed PM2.5
fig.add_trace(go.Scatter(
    x=combined.index,
    y=combined['Observed'],
    mode='lines+markers',
    name='Observed (PM2.5)',
    line=dict(color='royalblue', width=2)
))

# Forecast PM2.5
fig.add_trace(go.Scatter(
    x=combined.index,
    y=combined['Forecast'],
    mode='lines+markers',
    name='Forecast (PM2.5)',
    line=dict(color='orange', width=2, dash='dot')
))

# Layout and styling
fig.update_layout(
    title='🌫️ Observed vs Forecasted PM2.5 Concentration',
    xaxis_title='Datetime (UTC)',
    yaxis_title='PM2.5 (µg/m³)',
    legend=dict(x=0, y=1.1, orientation='h'),
    template='plotly_white'
)

fig.show()

# -------------------------------
# Step 4c: Compute Summary Metrics
# -------------------------------

# Only compute metrics where observed + forecast overlap
overlap = combined.dropna(subset=['Observed', 'Forecast'])

if len(overlap) > 0:
    mae = mean_absolute_error(overlap['Observed'], overlap['Forecast'])
    print("📊 Forecast Performance on overlapping period:")
    print(f"• Mean Absolute Error (MAE): {mae:.2f} µg/m³")
else:
    print("⚠️ No overlapping timestamps — MAE cannot be computed for future forecast.")

# Additional summary statistics for forecast
print("\n📈 Forecast Summary Statistics:")
print(f"• Average Forecast PM2.5: {forecast_df['pm25_pred'].mean():.2f} µg/m³")
print(f"• Minimum Forecast PM2.5: {forecast_df['pm25_pred'].min():.2f} µg/m³")
print(f"• Maximum Forecast PM2.5: {forecast_df['pm25_pred'].max():.2f} µg/m³")

# -------------------------------
# Step 4d: Optional Anomaly / High Pollution Check
# -------------------------------

# Example threshold for unhealthy PM2.5
threshold = 90
high_pollution = forecast_df[forecast_df['pm25_pred'] > threshold]

if not high_pollution.empty:
    print(f"🚨 Warning: {len(high_pollution)} forecasted hours exceed {threshold} µg/m³ (unhealthy level).")
    display(high_pollution)
else:
    print("✅ All forecasted PM2.5 levels are within safe limits.")


✅ Forecast completed! Generated 24 future PM2.5 predictions.


/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, but StandardScaler was fitted with feature names

/usr/local/lib/python3.12/dist-packages/sklearn/utils/validation.py:2739: UserWarning:

X does not have valid feature names, 

,pm25_pred
datetime,
2025-10-17 00:00:00,8.434589
2025-10-17 01:00:00,8.535718
2025-10-17 02:00:00,8.641415
2025-10-17 03:00:00,9.739295
2025-10-17 04:00:00,9.920545


✅ Combined observed and forecast data ready for visualization.


,Observed,Forecast
datetime,,
2025-10-17 19:00:00,NaN,9.346913
2025-10-17 20:00:00,NaN,9.454739
2025-10-17 21:00:00,NaN,9.503257
2025-10-17 22:00:00,NaN,9.422892
2025-10-17 23:00:00,NaN,9.382538


⚠️ No overlapping timestamps — MAE cannot be computed for future forecast.

📈 Forecast Summary Statistics:
• Average Forecast PM2.5: 9.26 µg/m³
• Minimum Forecast PM2.5: 8.43 µg/m³
• Maximum Forecast PM2.5: 9.92 µg/m³
✅ All forecasted PM2.5 levels are within safe limits.


In [27]:
# ============================================================
# 🧩 STEP 5: ANOMALY VISUALIZATION + EXPORT FORECAST
# ============================================================

import plotly.express as px

# -------------------------------
# 1️⃣ Define anomalies / high pollution
# -------------------------------

# Example: PM2.5 > 90 µg/m³ is considered unhealthy
threshold = 90

# Add a column for anomaly flag
forecast_df['anomaly'] = forecast_df['pm25_pred'] > threshold

# Count anomalies
num_anomalies = forecast_df['anomaly'].sum()
print(f"⚠️ Number of predicted high PM2.5 hours (> {threshold} µg/m³): {num_anomalies}")

# -------------------------------
# 2️⃣ Plot forecast with anomaly overlay
# -------------------------------

fig2 = go.Figure()

# Forecast line
fig2.add_trace(go.Scatter(
    x=forecast_df.index,
    y=forecast_df['pm25_pred'],
    mode='lines+markers',
    name='Forecast PM2.5',
    line=dict(color='orange', width=2)
))

# Highlight anomalies
anomalies = forecast_df[forecast_df['anomaly']]
if not anomalies.empty:
    fig2.add_trace(go.Scatter(
        x=anomalies.index,
        y=anomalies['pm25_pred'],
        mode='markers',
        name='High PM2.5 (> 90)',
        marker=dict(color='red', size=10, symbol='x')
    ))

fig2.update_layout(
    title='Forecasted PM2.5 with Anomaly Highlights',
    xaxis_title='Datetime (UTC)',
    yaxis_title='PM2.5 (µg/m³)',
    template='plotly_white'
)

fig2.show()

# -------------------------------
# 3️⃣ Export forecast to CSV
# -------------------------------

forecast_df.to_csv('pm25_forecast.csv', index=True)
print("✅ Forecast saved to 'pm25_forecast.csv'")


⚠️ Number of predicted high PM2.5 hours (> 90 µg/m³): 0


✅ Forecast saved to 'pm25_forecast.csv'


NameError: name 'model_pipeline' is not defined